In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import sys
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm

# Add the parent directory to sys.path
sys.path.append(os.path.dirname(os.path.realpath(os.path.abspath(""))))

from unet.dataset import DeadwoodDataset
from unet.dice_score import dice_loss
from unet.evaluate import evaluate
from unet.unet_model import UNet

In [3]:
register_file = "/net/scratch/jmoehring/tiles_register_biome_bin.csv"
register_df = pd.read_csv(register_file)

In [4]:
dataset = DeadwoodDataset(
    register_df=register_df,
    no_folds=2,
    random_seed=1,
    test_size=0,
)

In [5]:
loader_args = {
    "batch_size": 128,
    "num_workers": 64,
    "pin_memory": True,
    "shuffle": False,
}
train_set, _ = dataset.get_train_val_fold(0)
train_loader = DataLoader(train_set, **loader_args)

In [9]:
# Initialize counters
total_deadwood_pixels = 0
total_pixels = 0

# Iterate over batches
count = 0
for _, masks, _ in tqdm(train_loader, total=len(train_loader)):
    total_deadwood_pixels += masks.sum().item()  # Accumulate deadwood pixels
    total_pixels += masks.numel()  # Accumulate total pixels
    count += 1
    if count == 1000:
        break

# Calculate the overall ratio
overall_deadwood_ratio = total_deadwood_pixels / total_pixels

 23%|██▎       | 999/4347 [02:35<08:40,  6.43it/s] 


In [11]:
total_pixels / total_deadwood_pixels

42.81936271859704